In [63]:
import markdown 
from markdown.extensions import Extension
from markdown.preprocessors import Preprocessor
from markdown.postprocessors import Postprocessor
from markdown.treeprocessors import Treeprocessor
from xml.etree import ElementTree as etree
import uuid
import re

In [64]:
def random_id():
    return uuid.uuid4().hex[:10]

In [69]:

# Custom Postprocessor to add random IDs to HTML elements
class RandomIdPostprocessor(Postprocessor):
    def run(self, text):
        # Regular expression to match HTML tags without an id attribute
        def add_id_to_tag(match):
            tag = match.group(0)
            # Only add id if there's no existing id attribute
            if 'id="' not in tag:
                # Insert a random id into the tag
                return re.sub(r'(<\w+)', rf'\1 id="{random_id()}"', tag)
            return tag
        
        # Apply the function to each HTML tag in the text
        return re.sub(r'<\w+[^>]*>', add_id_to_tag, text)

# Extension to register the Postprocessor
class RandomIdExtension(Extension):
    def extendMarkdown(self, md):
        # Register the Postprocessor
        md.postprocessors.register(RandomIdPostprocessor(md), 'random_id_postprocessor', 25)


In [70]:
md = markdown.Markdown(extensions=[RandomIdExtension()])


In [71]:
text = """
# Heading 1

Some paragraph text.

## Heading 2

More text here.
"""

In [76]:
md.convertFile

<bound method Markdown.convertFile of <markdown.core.Markdown object at 0x000002A89AFC3310>>

In [24]:
folders = ['ovid', 'tacitus']

In [78]:
import os

for folder in folders:
    dir = '../'+folder
    files = os.listdir(dir)
    os.makedirs('../public/'+folder, exist_ok=True)
    folderMD = f"# {folder}\n\n"
    for fn in files:
        if fn.endswith('.txt'):
            ofn = fn[:-4] + '.html'
            ofile = '../public/' + folder + '/' + ofn
            ifile = dir +'/' + fn
            md.convertFile(input=ifile, output=ofile)
            folderMD += f"[{fn[:-4] + '.html'}]({folder + "/" + ofn})\n\n"
    with open('../public/' + folder +'.html', 'w') as f:
        f.write( md.convert(folderMD))